# CSMCC16 Flight Coursework

### Import packages

In [131]:
import numpy as np
import pandas as pd
import datetime
import re
from difflib import SequenceMatcher
import PySimpleGUI as sg
import math

### Import data

In [2]:
#import csv as dataframe
pass_data = pd.DataFrame(pd.read_csv('AComp_Passenger_data.csv',header=None))
airport_lat_lon = pd.DataFrame(pd.read_csv('Top30_airports_LatLong.csv',header=None))

In [3]:
# Add Column Names
pass_data.columns = ['Passenger_id'
                        ,'Flight_id'
                        ,'From_airport_IATA'
                        ,'Dest_airport_IATA'
                        ,'Depart_time_epoch'
                        ,'Flight_time_min']
airport_lat_lon.columns = ['Airport_name'
                        ,'Airport_IATA_code'
                        ,'Latitude'
                        ,'Longitude']

### Remove missing values

In [4]:
#Remove values where passenger id and flight id are missing
pass_data=pass_data.dropna(subset=['Flight_id','Passenger_id'])

### Functions for Additional Columns and formats

In [5]:
#calculate datetime values from epoch and arrival time
def calc_arrive_time(df):
    df['Depart_time_datetime'] = pd.to_datetime(df['Depart_time_epoch'],unit='s')
    df['Arrive_epoch'] = pass_data['Depart_time_epoch']+df['Flight_time_min']*60
    df['Arrive_time_datetime'] = pd.to_datetime(df['Arrive_epoch'],unit='s')

In [6]:
#create route (to use as unique id)
def route(df):
    df['Route'] = df['From_airport_IATA']+'-' + df['Dest_airport_IATA']+'-' + df['Depart_time']

In [7]:
#break route for details
def break_route(df):
    df[['From_airport_IATA','Dest_airport_IATA','Depart_time']]=df['Route'].str.split('-',expand=True)

In [8]:
#break_route(pass_data_route)
#pass_data_route[['From_airport_IATA','Dest_airport_IATA','Depart_time']]=pass_data_route['Route'].str.split('-',expand=True)

In [9]:
#Add new datetime formats
def add_times(df):
    df['Depart_day_month']=df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
    #df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
    df['Depart_time']=df['Depart_time_datetime'].dt.strftime("%H:%M:%S")
    df['Arrive_time']=df['Arrive_time_datetime'].dt.strftime("%H:%M:%S")
    df['Depart_day_month']=df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y")
    #pass_data_df['Depart_day_datetime']=pass_data_df['Depart_time_datetime'].dt.strftime("%a %d/%b/%Y %H:%M:%S")
    df['Depart_time']=df['Depart_time_datetime'].dt.strftime("%H:%M:%S")
    df['Arrive_time']=df['Arrive_time_datetime'].dt.strftime("%H:%M:%S")

In [10]:
#Add column to allow counting of passengers
def add_1(df):
    df['count']='1'

## Setting up the gui output

In [11]:
sg.SetOptions(
    background_color='#152238',
       text_element_background_color='#152238',
       element_background_color='#152238',
       text_color='#FFFFFF',
       scrollbar_color=None,
       input_elements_background_color='#FFFFFF',
       progress_meter_color = ('green', 'blue')
#        button_color=('white','#475841')
    )

True

### Add Additional Columns and formats

In [ ]:
# route(flight_only_data)
####check if necessary!!
add_1(pass_data)
# pass_data
route(pass_data)
# Select Route info only, and group data
key_route_info=['Route'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_day_month'
                      ,'Depart_time'
                      ,'Flight_time_min'
                      ,'Arrive_time']
route_info=pass_data[key_route_info]
route_info=route_info.groupby(key_route_info).count()
route_info.reset_index(inplace=True)

# def group(base_df,variables,new_df):
#     new_df=pd.DataFrame()
#     #select variables only
#     new_df=base_df[variables]
#     #group by variables to create a table
#     new_df=new_df.groupby(variables).count()
#     #reset index
#     new_df.reset_index(inplace=True)
    
# key_route_info=['Route'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_day_month'
#                       ,'Depart_time'
#                       ,'Flight_time_min'
#                       ,'Arrive_time']
# route_info_new=pd.DataFrame()
# group(pass_data,key_route_info,route_info_new)

# route_info

# Number of passengers per flight id


## Error Correction
### Flights

In [ ]:
# #select subset of data 
# flight_info = ['Flight_id'
#                       ,'From_airport_IATA'
#                       ,'Dest_airport_IATA'
#                       ,'Depart_time'         
#                         ]
# flight_only_data = pass_data[flight_info]
# #Group data by flight
# flight_only_data = flight_only_data.groupby(flight_info).count()
# #reset index
# flight_only_data.reset_index(inplace=True)


### Flight data only

In [12]:
#Add flight time info 
calc_arrive_time(pass_data)
add_times(pass_data)

In [30]:
#select subset of data 
flight_info = ['Flight_id'
                      ,'From_airport_IATA'
                      ,'Dest_airport_IATA'
                      ,'Depart_time'         
                        ]
flights = pass_data[flight_info]
#Group data by flight
flights = flights.groupby(flight_info).count()
#reset index
flights.reset_index(inplace=True)
#add route info
route(flights)
# flights

In [31]:
#check if airport in airport list
airports = airport_lat_lon['Airport_IATA_code']
# airports

In [32]:
#Not necessary but keeping for reference

#correct airport format
format_IATA = r"[A-Z][A-Z][A-Z]"
#remove row if from airport not correct format
flights=flights[flights['From_airport_IATA'].str.contains(format_IATA)]
#remove row if dest airport not correct format
flights=flights[flights['Dest_airport_IATA'].str.contains(format_IATA)]

In [33]:
#remove row if flight id not correct format
format_flight_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z]"
flights=flights[flights['Flight_id'].str.contains(format_flight_id)]

In [34]:
# Only keep data if airport is in list
flights=flights[flights['From_airport_IATA'].isin(airports)]
flights=flights[flights['Dest_airport_IATA'].isin(airports)]
# flights

In [35]:
# Calculate the number of passengers per flight
flight_pass = ['Flight_id','Passenger_id']
flight_pass_data = pass_data[flight_pass]
pass_per_flight=flight_pass_data.groupby('Flight_id').count()
pass_per_flight.reset_index(inplace=True)
#pass_per_flight

In [36]:
#merge correct flights with passenger numbers
flights_merge=flights.merge(pass_per_flight, how='inner', on=['Flight_id'])

In [37]:
# Remove duplicate routes with lowest number of passenger ids
flights_merge=flights_merge.sort_values(by=['Route','Passenger_id']
                             , ascending=[True,False]
                             , na_position='first')
flights_merge=flights_merge[flights_merge['Route'].duplicated()== False]
# remove column of counts
flights_final=flights_merge.drop(labels='Passenger_id', axis=1)
# flights_merge = flights_merge[flights_merge.columns[:-1]]
# flights_merge

In [ ]:
# #add route info to flight only data
# route(flight_only_data)

## add to passenger data

In [38]:
#Add route info to pass_data
route(pass_data)

In [39]:
#Select data required
pass_data_slim = pass_data[['Passenger_id','Flight_id','Route']]
flight_id_route = flights_final[['Flight_id','Route']]

In [44]:
# correct flight info
def correct_flight_errors(column1,column2):
    global pass_data_route
    pass_data_route = []
    for ind in pass_data_slim.index:
        # if column 1 is correct...
        if (pass_data[column1][ind] in flight_id_route[column1].to_list()) == True:
            # take the data from passenger data
            pass_data_route.append([pass_data_slim['Passenger_id'][ind]
                            ,pass_data_slim['Flight_id'][ind]
                            ,pass_data_slim['Route'][ind]])
        # if column 1 is NOT correct...
        else:
            #  look up the correct value
            true_route = (pass_data[column2][ind])
            #  get the row number of correct value
            row = flight_id_route[flight_id_route[column2] == true_route].index[0]
            #  take passenger id from passenger data, take other data from correct table
            pass_data_route.append([pass_data_slim['Passenger_id'][ind]
                            , flight_id_route['Flight_id'][row]
                            , flight_id_route['Route'][row]])
    pass_data_route= pd.DataFrame(pass_data_route, columns=['Passenger_id','Flight_id','Route'])

In [45]:
correct_flight_errors('Flight_id','Route')
correct_flight_errors('Route','Flight_id')

In [100]:
pass_data_route

,Passenger_id,Flight_id,Route,From_airport_IATA,Dest_airport_IATA,Depart_time
0,UES9151GS5,SQU6245R,DEN-FRA-17:14:20,DEN,FRA,17:14:20
1,UES9151GS5,XXQ4064B,JFK-FRA-17:05:17,JFK,FRA,17:05:17
2,EZC9678QI6,SOH3431A,ORD-MIA-17:00:49,ORD,MIA,17:00:49
3,ONL0812DH1,SOH3431A,ORD-MIA-17:00:49,ORD,MIA,17:00:49
4,CYJ0225CH1,PME8178S,DEN-PEK-17:13:29,DEN,PEK,17:13:29
...,...,...,...,...,...,...
495,BWI0520BG6,BER7172M,KUL-LAS-17:26:07,KUL,LAS,17:26:07
496,LLZ3798PE3,EWH6301Y,CAN-DFW-17:22:47,CAN,DFW,17:22:47
497,KKP5277HZ7,KJR6646J,IAH-BKK-17:26:43,IAH,BKK,17:26:43
498,JJM4724RF7,XXQ4064B,JFK-FRA-17:05:17,JFK,FRA,17:05:17


## Error Correction
### Passenger

In [ ]:
format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
pass_data_route_test=pass_data_route
pass_data_route_test['Pass_id_format'] =pass_data_route['Passenger_id'].str.contains(format_pass_id)
pass_data_route_test

In [ ]:
#add column to count
pass_data_route_test['count'] = '1'

In [ ]:
#select subset of data 
pass_data_route_test = pass_data_route_test[['Passenger_id'
                      ,'Pass_id_format'
                      ,'count'
                        ]]

In [ ]:
#Group data to see passengers and total flights by passenger id
pass_only=pass_data_route_test.groupby(['Passenger_id'
                        ,'Pass_id_format'
                         ]).count()
#reset index
pass_only.reset_index(inplace=True)
pass_only

In [ ]:
#select only those whose regex is correct 
## this is not necessary as the second code will remove these, 
## however it is left here as this will be a better way to check if correct passenger id
pass_info_true = pass_only.loc[pass_only['Pass_id_format'] == True]

In [ ]:
#select only those who have more than 1 flight
pass_info_true = pass_only.loc[pass_only['count'] != 1]
#pass_info_true

In [ ]:
pass_info_false = pass_only.loc[pass_only['count'] == 1]
#pass_info_false

In [ ]:
#Create list of false passenger ids
false = pass_info_false['Passenger_id']
false

In [ ]:
#Create list of true passenger ids
true = pass_info_true['Passenger_id']
#true

In [ ]:
#create function to find the best match for a given false value and list for true values
def best_match(f,t):
    global test_best_true
    global final_true
    test_best_true = []
    final_true =[]
    for t in true:
        test_best_true.append([t,round(SequenceMatcher(None, t, f).ratio(),3)])
    test_best_true = pd.DataFrame(test_best_true)
    max_true = test_best_true.iloc[test_best_true[1].idxmax(), 0]
    final_true.append([f,max_true])
    final_true=pd.DataFrame(final_true,columns=['false','true'])
   # print(final_true)

In [ ]:
# best_match(f,true)

In [ ]:
#test each false value
best_match_results = pd.DataFrame()
for f in false:
    best_match(f,true)
    best_match_results=best_match_results.append(final_true)

In [ ]:
best_match_results

## Objective 1: 
### Determine the number of flights from each airport; include a list of any airports not used.

In [47]:
airport_list = airport_lat_lon['Airport_IATA_code']
airport_list
break_route(pass_data_route)
from_list = pass_data_route['From_airport_IATA']

In [48]:
#mapper, where key = airport and value = 1 for each airport in from_list
key2val2 = [[a, 1] for a in from_list]
mapper = pd.DataFrame(key2val2, columns=['Airport_IATA_code','value'])
#mapper

#reducer - takes key and val from mapper to provide final key and value
reducer=mapper.groupby(['Airport_IATA_code']).sum()
reducer
reducer.reset_index(inplace=True)

In [49]:
# merge full list of airports with number of flights from each airport
airport_list=pd.DataFrame(airport_list)
num_flights = airport_list.merge(reducer, how='outer', on=['Airport_IATA_code'])

In [52]:
num_flights

,value
Airport_IATA_code,
DEN,46.0
CAN,37.0
IAH,37.0
ATL,36.0
KUL,33.0
ORD,33.0
CGK,27.0
JFK,25.0
LHR,25.0


In [53]:
# clean up results
num_flights=num_flights.fillna(0)
num_flights=num_flights.sort_values(by=['value','Airport_IATA_code'], ascending=[False,True])
# num_flights.set_index('Airport_IATA_code', inplace=True, drop=True)
# num_flights

## Objective 2
### Create a list of flights based on the Flight id, this output should include the passenger Id, relevant IATA/FAA codes, the departure time, the arrival time (times to be converted to HH:MM:SS format), and the flight times.

## Objective 3
### Calculate the number of passengers on each flight.

In [ ]:
# pass_data_route
# calc_arrive_time(pass_data_route)
#num_flights = airport_list.merge(from_counts_df, how='outer', on=['Airport_IATA_code'])

In [54]:
flights = pass_data_route['Flight_id']

In [55]:
#mapper, where key = airport and value = 1 for each airport in from_list
key2val2 = [[a, 1] for a in flights]
mapper = pd.DataFrame(key2val2, columns=['Flight_id','value'])
#mapper

#reducer - takes key and val from mapper to provide final key and value
num_passengers=mapper.groupby(['Flight_id']).sum()
#num_passengers
#reducer
num_passengers.reset_index(inplace=True)

In [56]:
# clean up results
#num_passengers=num_flights.fillna(0)
num_passengers=num_passengers.sort_values(by=['value','Flight_id'], ascending=[False,True])
num_passengers.set_index('Flight_id', inplace=True, drop=True)
num_passengers

,value
Flight_id,
ULZ8130D,27
GMO5938W,24
XXQ4064B,24
KJR6646J,23
QHU1140O,21
WSK1289Z,21
FYL5866L,20
JVY9791G,20
SQU6245R,20


## Objective 4
### Calculate the line-of-sight (nautical) miles for each flight and the total travelled by each passenger and thus output the passenger having earned the highest air miles.

In [ ]:
## NEED TO REDO!!

In [107]:
# pass_data_route (Passenger_id,Flight_id,Route,From_airport_IATA,Dest_airport_IATA,Depart_time)
# airport_lat_lon (Airport_IATA_code,Latitude,Longitude)

In [198]:
def calculate_miles(df_in):
    global miles_per_flight
    miles_per_flight = []
    for ind in df_in.index:
            #Take from airport
            p= df_in['Passenger_id'][ind]
            x= df_in['From_airport_IATA'][ind]
            #Find it in airport lat long list
            row = (airport_lat_lon[airport_lat_lon['Airport_IATA_code'] == x].index[0])
            #Give me the latitude and longitude
            a = airport_lat_lon['Latitude'][row]
            b = airport_lat_lon['Longitude'][row]
            #Take dest airport
            y = df_in['Dest_airport_IATA'][ind]
            #Find it in airport lat long list
            row = (airport_lat_lon[airport_lat_lon['Airport_IATA_code'] == y].index[0])
            #Give me the latitude and longitude
            c = airport_lat_lon['Latitude'][row]
            d = airport_lat_lon['Longitude'][row]
            miles = math.sqrt((c-a)**2+(d-b)**2)
            miles_per_flight.append([df_in['Passenger_id'][ind],miles])
    miles_per_flight= pd.DataFrame(miles_per_flight
                                   , columns=['Passenger_id','Miles']
                                  )    


In [199]:
calculate_miles(pass_data_route)

In [200]:
miles_per_flight
# pd.DataFrame(miles_per_flight)
# pass_data_route= pd.DataFrame(pass_data_route, columns=['Passenger_id','Flight_id','Route'

,Passenger_id,Miles
0,UES9151GS5,113.671693
1,UES9151GS5,82.855474
2,EZC9678QI6,17.886951
3,ONL0812DH1,17.886951
4,CYJ0225CH1,221.257842
...,...,...
495,BWI0520BG6,219.409177
496,LLZ3798PE3,210.551409
497,KKP5277HZ7,196.765308
498,JJM4724RF7,82.855474


In [201]:
key2val2 = pd.DataFrame()
for index,row in miles_per_flight.iterrows():
    key2val2=key2val2.append(miles_per_flight.loc[index])
    mapper = pd.DataFrame(key2val2, columns=['Miles','Passenger_id'])
mapper

,Miles,Passenger_id
0,113.671693,UES9151GS5
1,82.855474,UES9151GS5
2,17.886951,EZC9678QI6
3,17.886951,ONL0812DH1
4,221.257842,CYJ0225CH1
...,...,...
495,219.409177,BWI0520BG6
496,210.551409,LLZ3798PE3
497,196.765308,KKP5277HZ7
498,82.855474,JJM4724RF7


In [205]:
#reducer - takes key and val from mapper to provide final key and value
miles_per_pass=mapper.groupby(['Passenger_id']).sum().round(1)
#reducer
miles_per_pass.reset_index(inplace=True)

In [204]:
# clean up results
#num_passengers=num_flights.fillna(0)
miles_per_pass=miles_per_pass.sort_values(by=['Miles'], ascending=[False])
miles_per_pass.set_index('Passenger_id', inplace=True, drop=True)
miles_per_pass

,Miles
Passenger_id,
UES9151GS5,3378.5
DAZ3029XA0,3305.9
BWI0520BG6,3272.9
PUD8209OG3,3096.7
SPR4484HA6,2906.1
WYU2010YH8,2778.1
JJM4724RF7,2686.0
WBE6935NU3,2514.9
HCA3158QA6,2505.9
